In [1]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"


In [2]:
# Cameras
IMAGE_SIZE  = 240
CAMERAS = ['front', 'back']

# Scene
# [x_min, y_min, z_min, x_max, y_max, z_max] - the metric volume to be voxelized
SCENE_BOUNDS    = [-1.5, -1.5, -1.5, 1.50, 1.5, 1.5]

# Agents
AGENTS = ['Panda0', 'Panda1']

In [3]:
# load data
DATASET_PATH = '../outputs/dualarms_2cam/dualarms_2cam_with_fk.pkl'
with open(DATASET_PATH, 'rb') as f:
    _obs = pickle.load(f)

## Figure

In [ ]:
# show keys and shape
idx = -1
CAMERA = CAMERAS[0]
print('camera:', CAMERA)
print('keys:',_obs[idx][CAMERA].keys())
print('rgb',_obs[idx][CAMERA]['rgb'].shape)
print('depth',_obs[idx][CAMERA]['depth'].shape)
print('pcd',_obs[idx][CAMERA]['pointcloud'].shape)
print('resolution',_obs[idx][CAMERA]['resolution'])


# plot rgb and depth camera
fig = plt.figure(figsize=(10, 10))
rows, cols = 2, len(CAMERAS)
plot_idx = 1
for camera in CAMERAS:
    fig.add_subplot(rows, cols, plot_idx)
    plt.imshow(_obs[idx][camera]['rgb'])
    plt.axis('off')
    plt.title("%s_rgb" % (camera))

    fig.add_subplot(rows, cols, plot_idx+len(CAMERAS))
    plt.imshow(_obs[idx][camera]['depth'])
    plt.axis('off')
    plt.title("%s_depth" % (camera))
    plot_idx += 1
plt.show()

In [ ]:
def constrain_scene_bounds(pcd):
    global SCENE_BOUNDS
    pcd = pcd[:, ( (pcd[0,:] > SCENE_BOUNDS[0]) & (pcd[0,:] < SCENE_BOUNDS[3])  )] # x axis constrainsts
    pcd = pcd[:, ( (pcd[1,:] > SCENE_BOUNDS[1]) & (pcd[1,:] < SCENE_BOUNDS[4])  )] # y axis constrainsts
    pcd = pcd[:, ( (pcd[2,:] > SCENE_BOUNDS[2]) & (pcd[2,:] < SCENE_BOUNDS[5])  )] # z axis constrainsts
    return pcd


def constrain_scene_bounds2(pcd, rgb):
    global SCENE_BOUNDS

    # x axis constraints
    mask = ( (pcd[0,:] > SCENE_BOUNDS[0]) & (pcd[0,:] < SCENE_BOUNDS[3])  )
    pcd = pcd[:, mask] 
    rgb = rgb[:, mask]

    # y axis constraints
    mask = ( (pcd[1,:] > SCENE_BOUNDS[1]) & (pcd[1,:] < SCENE_BOUNDS[4])  )
    pcd = pcd[:, mask] 
    rgb = rgb[:, mask]
    
    # z axis constraints
    mask = ( (pcd[2,:] > SCENE_BOUNDS[2]) & (pcd[2,:] < SCENE_BOUNDS[5])  )
    pcd = pcd[:, mask] 
    rgb = rgb[:, mask]

    return pcd, rgb

In [ ]:
idx = -1

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}],],
    horizontal_spacing = 0.00,
    vertical_spacing = 0.00,
    subplot_titles=('front_pointcloud', 'back_pointcloud')
)

for i, camera in enumerate(CAMERAS):
    # print(camera)

    # get pointcloud
    pcd = _obs[idx][camera]['pointcloud'].transpose(2,0,1).reshape(3,-1)

    # get rgb
    rgb = _obs[idx][camera]['rgb'].transpose(2,0,1).reshape(3,-1)
    rgb = ((rgb - 0.0) * 255.0) / 1.0 + 0.0

    # constrain scene
    pcd, rgb = constrain_scene_bounds2(pcd, rgb)
    color = [f'rgb({rgb[0,i]},{rgb[1,i]},{rgb[2,i]})' for i in range(rgb.shape[1])]

    fig.add_trace(
        go.Scatter3d(
            x = pcd[0,:],
            y = pcd[1,:],
            z = pcd[2,:],
            mode='markers',
            marker=dict(
                size=2,
                color=color,
                # color=pcd[0,:],
                # colorscale='Turbo',
            ),
            name=camera
        ),
        row=1, 
        col=i+1,
    )

fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)

# fig.write_html('../outputs/dualarms_2cam/dualarms_2cam_with_fk_pc.html')

# fig.show


## Meshes and Bounding Boxes

In [4]:
ROOT = '/home/dev/ws_percept/src'

In [13]:
import os
import numpy as np
import cupoch as cph

path = os.path.join(ROOT, 'outputs/testdata/franka_panda/panda.urdf')
kin = cph.kinematics.KinematicChain(path)


geoms_list = list()

# FK for agent 0
agent = 'Panda0'
joint_map = dict()
for pos, val in enumerate(_obs[1][agent]['joint_positions']):
    joint_map[f'panda_joint{pos}'] = val
poses = kin.forward_kinematics(joint_map)
geoms = kin.get_transformed_visual_geometry_map(poses)
# add geometries
geoms_list += list(geoms.values())

# FK for agent 1
agent = 'Panda1'
joint_map = dict()
for pos, val in enumerate(_obs[1][agent]['joint_positions']):
    joint_map[f'panda_joint{pos}'] = val
poses = kin.forward_kinematics(joint_map)
geoms = kin.get_transformed_visual_geometry_map(poses)
# add geometries
geoms_list += list(geoms.values())


# cph.visualization.draw_geometries(geoms_list)


In [6]:
_obs[1]['Panda0']

{'joint_positions': [1.3875367641448975,
  1.3091490268707275,
  -1.5707720518112183,
  -0.6108652353286743,
  -4.76837158203125e-06,
  2.5307233333587646,
  1.3089821338653564],
 'global_position': array([-0.03395154, -0.74387997,  0.07001754]),
 'global_orientation': array([-1.57079649,  0.24409597, -1.57079637])}

In [ ]:
_obs[1]['Panda1']